In [1]:
%load_ext autoreload
%autoreload 2

import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from src.modules import constants as con
from src.modules import plotter as plttr
plt.style.use(['seaborn-paper','science','no-latex', 'std-colors'])
matplotlib.rc("font", family="Times New Roman")    
plts = []

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Parameter
tau = int(60*con.tau)

In [2]:
df = pd.read_csv("/usr/app/data/input/trippub.csv") 
df.head()

,HOUSEID,PERSONID,TDTRPNUM,STRTTIME,ENDTIME,TRVLCMIN,TRPMILES,TRPTRANS,TRPACCMP,TRPHHACC,...,OBHTNRNT,OBPPOPDN,OBRESDN,DTHTNRNT,DTPPOPDN,DTRESDN,DTEEMPDN,DBHTNRNT,DBPPOPDN,DBRESDN
0,30000007,1,1,1000,1015,15,5.244,3,0,0,...,20,750,300,50,750,300,350,30,300,300
1,30000007,1,2,1510,1530,20,5.149,3,0,0,...,30,300,300,50,1500,750,750,20,750,300
2,30000007,2,1,700,900,120,84.004,6,0,0,...,40,1500,750,50,1500,750,750,20,750,300
3,30000007,2,2,1800,2030,150,81.628,6,0,0,...,20,750,300,40,1500,750,750,40,1500,750
4,30000007,3,1,845,900,15,2.250,3,0,0,...,20,750,300,50,750,300,350,60,750,300


In [ ]:
df.shape

In [22]:
# Filter applied
# - TRPTRANS in [03, 04, 05, 06] || Mode of transportt (correspond to car, suv, van, pick up truck)
# - TRPHHVEH == '01' || Only if household vehicle is used
# - TRIPPURP in [HBO, HBSHOP, HBSCOREC, HBW]
# - TRPMILES must not be negative

# ?- Drop duplicates for combination HOUSEID + STRTTIME 

df_tmp = df[
        (df["TRPHHVEH"] == 1) 
        & (df["TRPTRANS"].isin([3, 4, 5, 6])) 
        & (df["TRIPPURP"].isin(["HBO", "HBSHOP", "HBSCOREC", "HBW"]))
        & (df["TRPMILES"] >= 0)
       ].copy()
df_tmp.shape


(423319, 115)

In [23]:
## Check average velocity
df_tmp["TRVLCMIN"] = df_tmp["TRVLCMIN"]/60


In [24]:
df_tmp = df_tmp.loc[df_tmp.TRPMILES <=40]
sum(df_tmp["TRPMILES"]/sum(df_tmp["TRVLCMIN"]))*1.60934

38.05195186980086

In [ ]:
# STRTTIME for deriving probabilitiy of starting trip at time
print(df_tmp["STRTTIME"].describe())
df_tmp["STRTTIME"] = df_tmp["STRTTIME"].astype("str").apply(lambda x: int(x) if len(x) < 3 else int(int(x[0:-2])*60+int(x[-2:])))                                                      
df_tmp["STRTTIME"].describe()

In [ ]:
g = sns.displot(df_tmp, x="STRTTIME", stat="probability",bins=list(range(0,1441, tau)), height = 7, aspect = 3)
plttr.formatPlot(g, "Time (min)" , "Probability","Tripstart Probability" , xticks = np.arange(0, 1441, 30))
plts += [("prob_trpstrt", g)]

In [ ]:
hist_strtttime = np.histogram(df_tmp["STRTTIME"], bins = list(range(0,1441, tau)), density = True)
d_strttime = pd.DataFrame(data = {"t": hist_strtttime[1][0:int(1441/tau)], "p(t)": hist_strtttime[0]*tau})
d_strttime[d_strttime["t"] > 400].head(10)

In [ ]:
d_strttime.to_pickle("/usr/app/data/probabilities/trpstrt.pkl") 

In [ ]:
# TRPMILES for each STRTIIME for deriving probability of Triplengt
df_tmp["TRPMILES"].describe()

In [ ]:
k = 1

## Ensure that if trip is started but length is in interval [0,1) length must be greater zero to account for right decision making
## Set each trip length to mean of intervalls (1km intervalls => +0.5)
bins = [b+0.5 for b in range(0,41,k)]

#range(0, int(max(df_tmp["TRPMILES"])), int(max(df_tmp["TRPMILES"])/k))
d_len = pd.DataFrame(columns=["t"]+bins[:-1])

for t in d_strttime["t"]:
    hist_len = np.histogram(df_tmp.loc[df_tmp.STRTTIME==t, "TRPMILES"], bins = bins, density = True)
    d_len.loc[t,:] = [t]+list(hist_len[0]*k)
    
d_len

In [ ]:
# Save distribution
d_len.to_pickle("/usr/app/data/probabilities/trplen.pkl") 

In [ ]:
# Just for checking
d_len.loc[:,'sum'] = d_len.iloc[:,1:].sum(axis=1)
#d_len.iloc[:,1:]
d_len[d_len["t"] == 300].tail(1)

In [ ]:
d_len.tail()

In [ ]:
pd.DataFrame([(p,np.percentile(df_tmp.loc[:, "TRPMILES"], p)) for p in range(90,101,1)])

### => 97% of trips have lenght <= 40.01500 miles

#percs = pd.DataFrame([(p,np.percentile(df_tmp.loc[:, "TRPMILES"], p)) for p in range(0,101,1)])
#percs.head()
#percs[0]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
g = sns.histplot(ax=ax, data=df_tmp["TRPMILES"],color = plt.rcParams['axes.prop_cycle'].by_key()['color'][0])
plttr.formatPlot(g, "Trip Miles" , "Amount","")
plts += [("prob_trpmiles_total", plt.gcf())]

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
g = sns.histplot(ax=ax, data=df_tmp[df_tmp["TRPMILES"]<50]["TRPMILES"],color = plt.rcParams['axes.prop_cycle'].by_key()['color'][0])
ax.axvline(np.percentile(df_tmp["TRPMILES"], 97), alpha = 1, ymax = 1, linestyle = ":", color="black")
ax.text(40.5, 10000, "97% Percentile", size = 12, alpha = 1, color = "black")
plttr.formatPlot(g, "Trip Miles" , "Amount","")
plts += [("prob_trpmiles_total_50", plt.gcf())]

In [ ]:
df_plot = pd.melt(d_len.iloc[:,:-1].copy(), id_vars=['t'], var_name = "Length", value_name = "d(t)")
df_plot["d(t)"] = df_plot["d(t)"]*100
df_plot["d(t)"] = df_plot["d(t)"].astype("int32")
df_pivot = df_plot.pivot("t", "Length", "d(t)")

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(20, 20))
g = sns.heatmap(df_pivot, annot=True, fmt="d", linewidths=.5, ax=ax,cmap = sns.color_palette("viridis", as_cmap=True))
plts += [("prob_trpln_heat", g)]

In [ ]:
g = sns.FacetGrid(df_plot, col="t", col_wrap = 6)
g.map_dataframe(sns.barplot, x="Length", y="d(t)", palette = sns.color_palette("hls", len(df_plot["Length"].unique())))
g.set_axis_labels("Distance", "d(t)")
plts += [("prob_trpstrt_t", g)]

In [ ]:
plt.figure(figsize=(15,8))
plt = sns.lineplot(x=df_plot["Length"], y=df_plot["d(t)"]/100,legend = False,
            )# hue=df_plot["strttime"], palette = sns.color_palette("hls", len(df_plot["strttime"].unique())))
plts += [("prob_trpln_agg", g)]

In [ ]:
# Read in excel
prc = pd.read_csv("/usr/app/data/input/Price_avg.csv", sep=";")
prc.drop(columns=["Stunde des Tages"], inplace=True)
# Calculate euro/kWh
prc[["Year","Winter","Sommer"]] = prc[["Year","Winter","Sommer"]]/1000
prc["Count"] = (prc["Count"]-1)*int(tau)
prc.head()

In [ ]:
def construct_pric_dist(prc):

    prc["mu"] = prc["Year"]
    prc["sd"] = abs(prc["Year"]-prc["Winter"])
    prob_ls = []
    for t in prc["Count"]:
        # Derive normal distribution for each point in time (sd is sqrt(1/2 * (mean-lower)^2 + (mean-upper)^2)) = mean-lower
        s = np.random.normal(prc.loc[prc["Count"] == t,"mu"], prc.loc[prc["Count"] == t,"sd"], 10000)

        # Derive histrogram from normal distribution with 3 bins
        hist, bin_edges = np.histogram(s, bins = 3, density = True)

        # Construct dataframe - use means of bin edges as price and corresponding hist probability
        prob = pd.DataFrame({"prc": [round((x+y)/2,3) for x,y in zip(bin_edges, bin_edges[1:])], "p" : hist * np.diff(bin_edges),
                            "mu": [prc.loc[prc["Count"] == t,"mu"].tolist()[0]]*3, "sd": [prc.loc[prc["Count"] == t,"sd"].tolist()[0]]*3}) 
        prob["t"] = t
        prob_ls += [prob]

    # Price to buy
    d_prc = pd.concat(prob_ls)
    return d_prc


In [ ]:
np.random.seed(0)
# Price to sell
d_prc_b = construct_pric_dist(prc)
print(d_prc_b.head())

# Price to sell
# Assume prices for selling are 10&% lower
prc[["Year","Winter","Sommer"]] = prc[["Year","Winter","Sommer"]]*0.9
d_prc_s = construct_pric_dist(prc)
print(d_prc_s.head())

In [ ]:
# Save distribution
d_prc_b[["prc","p","t"]] .to_pickle("/usr/app/data/probabilities/d_prc_b.pkl") 
d_prc_s[["prc","p","t"]].to_pickle("/usr/app/data/probabilities/d_prc_s.pkl") 

In [ ]:
# Aggregated plot with err

# Single plots
## Combine sell and buy
d_prc_b["type"] = "b"
d_prc_s["type"] = "s"
df_prc_plot = pd.concat([d_prc_b, d_prc_s]).drop(columns=["prc","p"]).drop_duplicates()
df_prc_plot.head()


In [ ]:
np.random.seed(0)
df_stacked = pd.DataFrame([[row["t"], row["type"]] + np.random.normal(row["mu"], row["sd"], 10000).tolist() for i, row in df_prc_plot.iterrows()],
                         columns=["t", "type"]+np.arange(10000).tolist())

df_stacked = pd.melt(df_stacked, id_vars=["t","type"], var_name = "count", value_name = "values")





In [ ]:
# Not nice as bars set at one price point for both buy and sell... that is not true acutally
g = sns.FacetGrid(pd.concat([d_prc_b, d_prc_s]), col="t", hue="type", col_wrap = 6, sharex=False)
g.map_dataframe(sns.barplot, x="prc", y="p",alpha=0.5)
g.set_axis_labels("Price", "d(p)")

In [ ]:
# Buy
g = sns.FacetGrid(d_prc_b.loc[d_prc_b["t"].isin([480,1080,1410])], col="t", col_wrap = 3, sharex=False)
g.map_dataframe(sns.barplot, x="prc", y="p")
g.set_axis_labels("Price", "Probability")
for ax in g.axes.flat:
    ax.set_xticklabels([round(float(t.get_text()), 3)  for t in ax.get_xticklabels()])

plts += [("prob_prc_b_t", g)]

In [ ]:
# Sell
g = sns.FacetGrid(d_prc_s.loc[d_prc_b["t"].isin([480,1080,1410])], col="t", col_wrap = 3, sharex=False)
g.map_dataframe(sns.barplot, x="prc", y="p")
g.set_axis_labels("Price", "Probability")
for ax in g.axes.flat:
    ax.set_xticklabels([round(float(t.get_text()), 3)  for t in ax.get_xticklabels()])

plts += [("prob_prc_s_t", g)]

In [ ]:
# Combined prices
fig, ax = plt.subplots(figsize=(9, 6))
g = sns.lineplot(ax=ax, data=pd.concat([d_prc_b, d_prc_s]), x="t", y="prc", hue="type", legend = False)
plttr.formatPlot(g, "Time (min)" , "Price (€/kWh)","Buying and Selling Prices with Error Bounds" , "Price Type", ['Buying', 'Selling'], xticks = np.arange(0, 1441, 60))
plts = [("prob_prc_t", plt.gcf())]

In [ ]:
plts

In [ ]:
# Export plots
plttr.save(plts)